# A Learning Perspective on Random-Order Covering Problems

The term "learning" in the context of the **ROSC-OCO** (Random-Order Set Cover using Online Convex Optimization) algorithm is incorporated through the **Online Convex Optimization (OCO)** framework, specifically via the **Online Mirror Descent (OMD)** update rule.

The algorithm uses **learning** to dynamically adjust its strategy (which sets to purchase) based on the cost and coverage requirements observed *so far* in the sequence of arriving elements.

Here is a breakdown of how this learning mechanism works:

### 1. The Decision Variable ($\mathbf{p}^t$)

* **What it is:** The algorithm maintains a fractional vector $\mathbf{p}^t$, where each component, $p_S^t$, is a weight assigned to set $S$ at time $t$. This vector represents the algorithm's learned **"propensity to select"** that set.
* **Budget Constraint (The "Lesson"):** Crucially, the weights are constrained by the estimated optimal cost ($EST$) via a **cost constraint** $\sum_{S} c(S) \cdot p_S^t \leq EST$. The OCO framework learns a policy that ensures the total fractional cost of the chosen sets remains within this budget.

### 2. The Learning Signal ($H^t$ - The Stochastic Subgradient)

At each time step $t$, when a new element $e^t$ arrives, the algorithm receives feedback that drives the learning:

* **Gain Function:** The algorithm implicitly tries to maximize a *gain* function related to covering the elements.
* **Stochastic Subgradient ($H^t$):** If the element $e^t$ is currently uncovered, a **stochastic subgradient** vector, $H^t$, is computed. For any set $S$ that covers $e^t$, $H^t_S$ is set to a positive value (proportional to $\kappa_{e^t}/EST$, where $\kappa_{e^t}$ is the cost of the cheapest set covering $e^t$).
* **Interpretation:** $H^t$ acts as a **reward signal** for sets that could have covered the newly arrived, uncovered element. A higher $H^t_S$ means set $S$ was "more useful" in the current step.

### 3. The Learning Rule (OMD Update)

The $H^t$ signal is fed into the **Online Mirror Descent (OMD)** rule (a generalization of the Multiplicative Weights Update rule), which "learns" the next weight vector $\mathbf{p}^{t+1}$:

$$\mathbf{p}^{t+1} \leftarrow \text{OMD-Update}(\mathbf{p}^t, H^t)$$

This update performs two actions simultaneously:

1.  **Increases Weights:** It exponentially increases the weights ($p_S^t$) of sets $S$ that received a high positive reward $H^t_S$ (i.e., sets that covered the uncovered element $e^t$).
2.  **Projects Onto Budget:** It mathematically projects the new weight vector onto the feasible set defined by the constraints ($\mathbf{p} \in [0, 1]^m$ and $\sum c(S) \cdot p_S \leq EST$). This ensures the learned policy remains within the budget.

### Summary

The **ROSC-OCO** algorithm doesn't use machine learning in the modern deep learning sense. Instead, it uses **algorithmic learning** via the **OCO/OMD framework** to transform the online optimization problem into a repeated game against an adversary (the sequence of element arrivals). The competitive ratio guarantee relies on the OMD rule's proven ability to track the best fixed policy in hindsight, which, in this context, translates to an effective set cover strategy.

In [4]:
import random
import numpy as np

# --- Data Structure for Set Cover Problem ---

class SetCoverInstance:
    """Represents a Weighted Set Cover instance."""
    def __init__(self, U, S, costs, incidence):
        """
        :param U: List of elements (e.g., [1, 2, 3])
        :param S: List of set IDs (e.g., ['A', 'B', 'C'])
        :param costs: Dictionary mapping set ID to its cost.
        :param incidence: Dictionary mapping element to list of set IDs that cover it.
        """
        self.U = U
        self.S = S
        self.costs = costs
        self.incidence = incidence
        self.m = len(S)
        self.n = len(U)

        # Precalculate kappa_e: cost of the cheapest set containing element e
        self.kappa = {}
        for e in U:
            min_cost = float('inf')
            cheapest_set = None
            for set_id in incidence.get(e, []):
                cost = costs[set_id]
                if cost < min_cost:
                    min_cost = cost
                    cheapest_set = set_id
            self.kappa[e] = min_cost if cheapest_set else float('inf')
    
    def get_cheapest_set(self, e):
        """Returns the set ID that covers e with cost kappa[e]."""
        min_cost = self.kappa.get(e, float('inf'))
        for set_id in self.incidence.get(e, []):
            if self.costs[set_id] == min_cost:
                return set_id
        return None

# --- ROSC-OCO Algorithm (Algorithm 1) ---

class ROSC_OCO_Algorithm:
    """
    Implements the ROSC-OCO algorithm (Algorithm 1) for random-order weighted set cover.
    """
    def __init__(self, instance, EST):
        self.instance = instance
        self.EST = EST
        self.eta = 0.5  # OCO parameter, used as 1/2 in the paper (Theorem 2.2, Line 1929)
        self.all_elements = list(instance.U)
        
        # OCO state: p^t, a vector p in [0, 1]^m (indexed by set ID)
        self.p = {set_id: 0.0 for set_id in instance.S}
        
        # Algorithm state: current set of chosen resources R
        self.R_chosen = set()
        
        # Track covered elements
        self.covered_elements = set()
        
        # Statistics
        self.total_cost = 0.0
        self.history = []

    def _is_covered(self, e):
        """Checks if an element e is covered by a set in R_chosen."""
        return e in self.covered_elements
        
    def OMD_Update(self, p_t, H_t):
        """
        Conceptual OCO-ALG step (Line 6 of Algorithm 1).
        This implements a simplified Multiplicative Weights Update (MWU)
        for *gain* maximization, followed by a [0, 1] projection.
        The full OMD step for the scaled simplex A is complex and omitted.

        :param p_t: Current weights vector (p^t)
        :param H_t: Stochastic subgradient (H_t)
        :return: Next weights vector (p^{t+1})
        """
        p_next = {}
        for set_id in self.instance.S:
            # MWU step for gain maximization (p * exp(eta * gain_grad))
            # H_t is the stochastic gradient of the gain function
            unconstrained_p = p_t[set_id] * np.exp(self.eta * H_t.get(set_id, 0.0))
            
            # Projection: Enforce p_S in [0, 1] constraint. 
            # The complex cost constraint <c, p> <= EST is implicitly assumed 
            # to be handled by the black-box OCO-ALG from the paper.
            p_next[set_id] = min(unconstrained_p, 1.0)
            
        return p_next

    def run_simulation(self):
        """
        Simulates the ROSC-OCO algorithm assuming a random arrival order.
        """
        # Element arrival is in uniformly random order
        arrival_order = list(self.all_elements)
        random.shuffle(arrival_order)
        
        # U^t is the set of elements not seen until time t
        U_t = set(self.all_elements)
        
        for t, e_t in enumerate(arrival_order):
            # Element e^t is the uniformly random element of U^t (Line 3)
            
            e_uncovered = not self._is_covered(e_t)
            
            # 1. Calculate kappa_e^t (Line 4)
            kappa_e_t = self.instance.kappa[e_t] if e_uncovered else 0.0
            
            H_t = {set_id: 0.0 for set_id in self.instance.S}
            current_cost = 0.0
            
            # 2. Cover (Line 4)
            if e_uncovered:
                # Add the cheapest set (paying kappa_e^t)
                cheapest_set = self.instance.get_cheapest_set(e_t)
                if cheapest_set and cheapest_set not in self.R_chosen:
                    self.R_chosen.add(cheapest_set)
                    current_cost += self.instance.costs[cheapest_set]
            
            # 3. Sampling (Line 5)
            # Probability for set S: q_S = (kappa_e^t / EST) * p_S^t
            sampling_sets = []
            if kappa_e_t > 0:
                for set_id in self.instance.S:
                    prob_sample = (kappa_e_t / self.EST) * self.p[set_id]
                    # Cap probability at 1.0, though p_S in [0,1] and kappa_e_t <= EST implies prob_sample <= 1
                    prob_sample = min(prob_sample, 1.0) 
                    
                    if random.random() < prob_sample:
                        if set_id not in self.R_chosen:
                            self.R_chosen.add(set_id)
                            current_cost += self.instance.costs[set_id]
                        sampling_sets.append(set_id)
                
            # 4. OCO Update (Line 6)
            if kappa_e_t > 0:
                # Compute the term for min(1, sum p_S) (required for subgradient)
                p_sum_over_sets_of_e = sum(self.p[s] for s in self.instance.incidence.get(e_t, []))
                
                if p_sum_over_sets_of_e < 1.0:
                    # H_t,S = (kappa_e^t / EST) * I(e^t in S) (Claim 3.5, simplified)
                    for set_id in self.instance.incidence.get(e_t, []):
                        H_t[set_id] = kappa_e_t / self.EST
                # If sum p_S >= 1, the min(1,...) is tight, and the subgradient is 0.
            
            self.p = self.OMD_Update(self.p, H_t)
            
            # 5. Update Total Cost and Covered Status
            self.total_cost += current_cost
            
            # Check coverage by the newly chosen set R_chosen (includes Line 4 and 5 additions)
            newly_covered = set()
            for s_id in self.R_chosen:
                # Need a reverse mapping for elements covered by a set.
                # Since the reverse mapping is not explicitly in the input, 
                # we skip the full coverage check here, but in a real-world model
                # this would involve checking all elements covered by newly added sets.
                # For simulation, we only update e_t's covered status for the next step.
                pass 
                
            if e_t not in self.covered_elements:
                 if any(e_t in self.instance.incidence.get(e_prime, []) for e_prime in self.covered_elements):
                    # In a proper model, we check if e_t is covered by a set in R_chosen.
                    # R_chosen is the union of sets chosen over all time.
                    # Since Line 4 explicitly adds a set to cover e_t if needed, e_t IS covered.
                    self.covered_elements.add(e_t)

            # 6. Update U^t (Line 7)
            # U^t is implicitly updated by removing the element from the list of arrivals.
            U_t.remove(e_t) 
            
            self.history.append({
                'time': t + 1,
                'element': e_t,
                'cost_incurred': current_cost,
                'total_cost': self.total_cost,
                'kappa_et': kappa_e_t,
                'sets_bought': self.R_chosen.copy()
            })
            
            if self.total_cost >= 2 * self.EST and self.EST != float('inf'):
                # Stopping condition based on potential Phi^t < EST (Line 1943), 
                # simplified here by checking total cost against a fixed EST.
                # The paper's stopping time tau is when the potential drops below EST.
                pass 
        
        print(f"Simulation Complete. Total Cost: {self.total_cost:.2f}")

# --- Example Usage (Conceptual Data) ---

# Example data:
# Elements U: e1, e2, e3
# Sets S: A, B, C
# Costs c: A=10, B=10, C=5
# Incidence:
# e1 is covered by {A}
# e2 is covered by {B}
# e3 is covered by {A, C}
# OPT cost: min(cost(A) + cost(B) = 20, cost(A) + cost(C) + cost(B) or similar)
# OPT fractional solution (e.g., xA=1, xB=1, xC=0) -> cost 20.

U_example = [1, 2, 3]
S_example = ['A', 'B', 'C']
costs_example = {'A': 10, 'B': 10, 'C': 5}
incidence_example = {
    1: ['A'], # kappa_1 = 10
    2: ['B'], # kappa_2 = 10
    3: ['A', 'C'] # kappa_3 = 5 (from set C)
}
# Initial Potential Phi^0 = kappa_1 + kappa_2 + kappa_3 = 10 + 10 + 5 = 25 [cite: 1940]
# Assume OPT cost c(OPT) = 20 (Set A and B cover everything)
# We choose EST such that c(OPT) <= EST <= 2*c(OPT). Let EST = 30.
EST_example = 30

instance = SetCoverInstance(U_example, S_example, costs_example, incidence_example)
algorithm = ROSC_OCO_Algorithm(instance, EST_example)

# Run the algorithm
print("Starting ROSC-OCO simulation...")
algorithm.run_simulation()
print("Final Weights p:", algorithm.p)

# Note on the implementation:
# The core complexity of ROSC-OCO lies in the OCO-ALG's projection onto 
# the scaled simplex A = {p in [0, 1]^m | <c, p> <= EST}, which is a non-trivial step,
# not provided explicitly in the paper's main text[cite: 1925]. The OMD_Update 
# above is a conceptual representation based on the MWU/OMD principles of the paper.

Starting ROSC-OCO simulation...
Simulation Complete. Total Cost: 25.00
Final Weights p: {'A': 0.0, 'B': 0.0, 'C': 0.0}


# implementation from the 1st set cover problem using ROSCO

# what is ROSCO The acronym 
ROSC-OCO stands for Random-Order Set Cover using Online Convex Optimization.
It is an online, randomized algorithm designed to solve the Weighted Set Cover problem when elements arrive in a random, permuted order

In [7]:
import math
import random
import numpy as np
from typing import Dict, Set, List, Tuple, Union

# ==============================================================================
# --- 1. Data Setup (Using User's Sets) ---
# ==============================================================================

# Input Data
S_list: List[Set[int]] = [
    {1, 2, 3, 4},       # S1
    {5, 6, 7, 8},       # S2
    {1, 9, 10},         # S3
    {2, 4, 6, 8, 10},   # S4
    {3, 5, 7, 9}        # S5
]
Costs: Dict[frozenset[int], float] = {
    frozenset({1, 2, 3, 4}): 0.8,
    frozenset({5, 6, 7, 8}): 0.2,
    frozenset({1, 9, 10}): 0.9,
    frozenset({2, 4, 6, 8, 10}): 1.0,
    frozenset({3, 5, 7, 9}): 0.5
}

# Derived Data Structures for both algorithms
INITIAL_ELEMENTS: Set[int] = set.union(*S_list)
U_static: List[int] = sorted(list(INITIAL_ELEMENTS))
SET_IDS: List[str] = [f'S{i+1}' for i in range(len(S_list))]

# Unified SETS data structure for SetCoverInstance compatibility
STATIC_SETS_DATA: Dict[str, Tuple[Set[int], float]] = {}
for i, s in enumerate(S_list):
    set_id = SET_IDS[i]
    cost = Costs[frozenset(s)] 
    STATIC_SETS_DATA[set_id] = (s, cost)

# --- Data for ROSC-OCO Algorithm ---
ROSC_COSTS: Dict[str, float] = {s_id: data[1] for s_id, data in STATIC_SETS_DATA.items()}
ROSC_INCIDENCE: Dict[int, List[str]] = {e: [] for e in U_static}
for s_id, (elements, _) in STATIC_SETS_DATA.items():
    for e in elements:
        ROSC_INCIDENCE[e].append(s_id)

# ==============================================================================
# --- 2. Shared/Static Set Cover Class ---
# ==============================================================================

class SetCoverInstance:
    """Represents a Weighted Set Cover instance, adapted for both contexts."""
    def __init__(self, U: Union[Set[int], List[int]], S_data: Dict[str, Tuple[Set[int], float]], costs: Dict[str, float] = None, incidence: Dict[int, List[str]] = None):
        if isinstance(U, Set):
             self.U = list(U)
        else:
            self.U = U
        self.n = len(self.U)
        
        # Static/Greedy-specific structures
        self.S_data = S_data # {set_id: (elements, cost)}
        
        # ROSC-OCO specific structures (if provided)
        self.S_ids = list(S_data.keys())
        self.costs = costs if costs is not None else {s_id: data[1] for s_id, data in S_data.items()}
        self.incidence = incidence if incidence is not None else self._build_incidence(S_data)
        
        self._calculate_kappa()

    def _build_incidence(self, S_data: Dict[str, Tuple[Set[int], float]]) -> Dict[int, List[str]]:
        incidence = {e: [] for e in self.U}
        for s_id, (elements, _) in S_data.items():
            for e in elements:
                incidence[e].append(s_id)
        return incidence

    def _calculate_kappa(self):
        """Precalculate kappa_e: cost of the cheapest set containing element e."""
        self.kappa = {}
        for e in self.U:
            min_cost = float('inf')
            cheapest_set = None
            for set_id in self.incidence.get(e, []):
                cost = self.costs[set_id]
                if cost < min_cost:
                    min_cost = cost
                    cheapest_set = set_id
            self.kappa[e] = min_cost if cheapest_set else float('inf')
    
    def get_cheapest_set(self, e: int) -> Union[str, None]:
        """Returns the set ID that covers e with cost kappa[e]."""
        min_cost = self.kappa.get(e, float('inf'))
        for set_id in self.incidence.get(e, []):
            if self.costs[set_id] == min_cost:
                return set_id
        return None

# ==============================================================================
# --- 3. Static Greedy Algorithm (Basis for Recourse Analysis) ---
# ==============================================================================

def static_greedy_set_cover(instance: SetCoverInstance) -> Tuple[Set[str], float]:
    """Implements the classic log n-approximation greedy algorithm."""
    U_uncovered = set(instance.U)
    S_chosen = set()
    total_cost = 0.0
    
    print("Starting Static Greedy Algorithm...")

    while U_uncovered:
        best_set = None
        min_cost_effectiveness = float('inf')
        
        for set_id, (elements, cost) in instance.S_data.items():
            if set_id in S_chosen:
                continue

            newly_covered = elements.intersection(U_uncovered)
            num_newly_covered = len(newly_covered)

            if num_newly_covered > 0:
                cost_effectiveness = cost / num_newly_covered
                
                if cost_effectiveness < min_cost_effectiveness:
                    min_cost_effectiveness = cost_effectiveness
                    best_set = set_id
        
        if best_set is None:
            break

        S_chosen.add(best_set)
        total_cost += instance.costs[best_set]
        
        newly_covered_by_best_set = instance.S_data[best_set][0].intersection(U_uncovered)
        U_uncovered.difference_update(newly_covered_by_best_set)
        
        print(f"  --> Selected {best_set} (Cost: {instance.costs[best_set]:.2f}, Covers: {len(newly_covered_by_best_set)}) | Uncovered remaining: {len(U_uncovered)}")

    return S_chosen, total_cost

# ==============================================================================
# --- 4. ROSC-OCO Algorithm (Online Set Cover) ---
# ==============================================================================

class ROSC_OCO_Algorithm:
    """
    Implements the ROSC-OCO algorithm (Algorithm 1) for random-order weighted set cover.
    This is an online, randomized algorithm, distinct from the main paper's dynamic algorithm.
    """
    def __init__(self, instance: SetCoverInstance, EST: float):
        self.instance = instance
        self.EST = EST 
        self.eta = 0.5  # OCO parameter
        self.all_elements = list(instance.U)
        
        self.p = {set_id: 0.0 for set_id in instance.S_ids} # OCO state: weight vector p^t
        self.R_chosen = set() # Algorithm state: current set of chosen resources R
        self.covered_elements = set()
        self.total_cost = 0.0
        self.history = []

    def OMD_Update(self, p_t: Dict[str, float], H_t: Dict[str, float]) -> Dict[str, float]:
        """
        Conceptual OCO-ALG step (Line 6 of Algorithm 1) using simplified MWU.
        The complex projection onto {p in [0, 1]^m | <c, p> <= EST} is omitted.
        """
        p_next = {}
        for set_id in self.instance.S_ids:
            # MWU step for gain maximization: p * exp(eta * gain_grad)
            unconstrained_p = p_t[set_id] * np.exp(self.eta * H_t.get(set_id, 0.0))
            
            # Projection: Enforce p_S in [0, 1] constraint.
            p_next[set_id] = min(unconstrained_p, 1.0)
            
        return p_next

    def run_simulation(self):
        """Simulates the ROSC-OCO algorithm assuming a random arrival order."""
        arrival_order = list(self.all_elements)
        random.shuffle(arrival_order)
        
        print("\nStarting ROSC-OCO Online Simulation (Random Order):")

        for t, e_t in enumerate(arrival_order):
            e_uncovered = e_t not in self.covered_elements
            current_cost_incurred = 0.0
            H_t = {set_id: 0.0 for set_id in self.instance.S_ids}
            
            kappa_e_t = self.instance.kappa.get(e_t, 0.0) if e_uncovered else 0.0
            
            # 1. Cover (Line 4) - Add the cheapest set if e_t is uncovered.
            if e_uncovered:
                cheapest_set = self.instance.get_cheapest_set(e_t)
                if cheapest_set and cheapest_set not in self.R_chosen:
                    self.R_chosen.add(cheapest_set)
                    current_cost_incurred += self.instance.costs[cheapest_set]
            
            # 2. Sampling (Line 5) - Sample other sets based on OCO weights p.
            if kappa_e_t > 0:
                for set_id in self.instance.S_ids:
                    prob_sample = (kappa_e_t / self.EST) * self.p[set_id]
                    prob_sample = min(prob_sample, 1.0) # Ensure probability is <= 1.0
                    
                    if random.random() < prob_sample:
                        if set_id not in self.R_chosen:
                            self.R_chosen.add(set_id)
                            current_cost_incurred += self.instance.costs[set_id]
            
            # 3. OCO Update (Line 6) - Update weights p for the next step.
            if kappa_e_t > 0:
                p_sum_over_sets_of_e = sum(self.p[s] for s in self.instance.incidence.get(e_t, []))
                
                if p_sum_over_sets_of_e < 1.0:
                    # H_t,S is the subgradient, proportional to kappa_e^t / EST
                    for set_id in self.instance.incidence.get(e_t, []):
                        H_t[set_id] = kappa_e_t / self.EST
                
            self.p = self.OMD_Update(self.p, H_t)
            
            # 4. Update Total Cost and Covered Status
            self.total_cost += current_cost_incurred
            
            # For simplicity, we assume e_t is covered after this step if uncovered before (due to Line 4)
            if e_uncovered and e_t not in self.covered_elements:
                 self.covered_elements.add(e_t)

            print(f"Time {t+1}: Element {e_t} arrived. Cost: {current_cost_incurred:.2f}. Total Cost: {self.total_cost:.2f}. Cover Size: {len(self.R_chosen)}")

        print(f"\nSimulation Complete. Total Cost: {self.total_cost:.2f}")

# ==============================================================================
# --- 5. Execution ---
# ==============================================================================

# Create a unified instance
unified_instance = SetCoverInstance(
    U=INITIAL_ELEMENTS, 
    S_data=STATIC_SETS_DATA, 
    costs=ROSC_COSTS, 
    incidence=ROSC_INCIDENCE
)

# --- A. Static Greedy Execution ---
print("="*60)
print("PART A: Static Greedy Set Cover (Approximation Basis for Recourse Analysis)")
print("="*60)

S_static, cost_static = static_greedy_set_cover(unified_instance)

print("-" * 40)
print(f"Final Sets Chosen: {S_static}")
print(f"Total Cost: {cost_static:.2f}")
print(f"Greedy Approx. Factor (H_n): {math.log(unified_instance.n) + 0.577:.2f} (approx. ln n)")
print("="*60)


# --- B. ROSC-OCO Execution ---

# Estimate of OPT: Use the cost of the static greedy solution as a rough upper bound,
# but the algorithm typically assumes a value EST such that OPT <= EST <= 2*OPT.
# For demonstration, we'll use a value slightly higher than the greedy cost.
EST_value = cost_static * 1.5 
print(f"\nPART B: ROSC-OCO Online Set Cover Simulation (Algorithm 1)")
print(f"Estimated OPT (EST) for OCO: {EST_value:.2f} (Based on 1.5 * Greedy Cost)")
print("="*60)

# Reset random seeds for repeatability of this simulation (though it's still randomized)
random.seed(42)
np.random.seed(42)

rosc_oco_algorithm = ROSC_OCO_Algorithm(unified_instance, EST=EST_value)
rosc_oco_algorithm.run_simulation()

print("="*60)

PART A: Static Greedy Set Cover (Approximation Basis for Recourse Analysis)
Starting Static Greedy Algorithm...
  --> Selected S2 (Cost: 0.20, Covers: 4) | Uncovered remaining: 6
  --> Selected S1 (Cost: 0.80, Covers: 4) | Uncovered remaining: 2
  --> Selected S3 (Cost: 0.90, Covers: 2) | Uncovered remaining: 0
----------------------------------------
Final Sets Chosen: {'S2', 'S3', 'S1'}
Total Cost: 1.90
Greedy Approx. Factor (H_n): 2.88 (approx. ln n)

PART B: ROSC-OCO Online Set Cover Simulation (Algorithm 1)
Estimated OPT (EST) for OCO: 2.85 (Based on 1.5 * Greedy Cost)

Starting ROSC-OCO Online Simulation (Random Order):
Time 1: Element 8 arrived. Cost: 0.20. Total Cost: 0.20. Cover Size: 1
Time 2: Element 4 arrived. Cost: 0.80. Total Cost: 1.00. Cover Size: 2
Time 3: Element 3 arrived. Cost: 0.50. Total Cost: 1.50. Cover Size: 3
Time 4: Element 9 arrived. Cost: 0.00. Total Cost: 1.50. Cover Size: 3
Time 5: Element 6 arrived. Cost: 0.00. Total Cost: 1.50. Cover Size: 3
Time 6: Ele

# References
arxiv.org/pdf/2511.07283